从两个渠道下载上市公司年报

- 优点==》数据全，所有历史的年报都有
- 缺点==》下载速度慢，PDF格式，导致有些下载文件不能提取出文本来


# 深交所和上交所股票代码的区别
>股票代码用数字表示股票的不同含义。股票代码除了区分各种股票，也有其潜在的意义。沪市股票买卖的代码是以600或601打头，如：运盛实业：股票代码是600股票代码767。中国国航：股票代码是601111。B股买卖的代码是以900打头，如：上电B股：代码是900901。深市A股票买卖的代码是以000打头，如：顺鑫农业：股票代码是000860。B股买卖的代码是以200打头，如：深中冠B股，代码是200018。

>沪市新股申购的代码是以730打头。如：中信证券：申购的代码是730030。深市新股申购的代码与深市股票买卖代码一样，如：中信证券在深市市值配售代码是003030。配股代码，沪市以700打头，深市以080打头。如：运盛实业配股代码是700767。深市草原兴发配股代码是080780。中小板股票代码以002打头，如：东华合创股票代码是002065。创业板股票代码以300打头，如：探路者股票代码是：300005


# 深圳证券交易所：
http://www.szse.cn/disclosure/listed/notice/index.html


# 上海深圳交易所
http://www.sse.com.cn/disclosure/listedinfo/announcement/





In [ ]:
# -*- coding: utf-8 -*-
import re
import sys
import re

str_pattern = "中国|美国|韩国"

pattern = re.compile(str_pattern)

str_to_match = "我是中国人"

m = pattern.search(str_to_match)
print(m)



In [ ]:
#!jupyter nbconvert --to script web_spider_sse_szse_pdf_urls.i
import requests
import re
yearReport_regex = re.compile('摘要|取消|英文|补充公告|关于调整|的公告')
def checkIsChsYearReport(fileName):
    flag = True
    if yearReport_regex.search(fileName): # 此处用用match，match不支持中文
        flag = False
    if not(fileName.find('年报')>=0 or fileName.find('年度报告') >=0 ):
        flag = False
    if flag:
        print('ok',' '*20,fileName)
    else:
        print('not ok',' '*16,fileName)
    return flag
    
    
def get_url_by_code_szse(code):
    download_url_prefix = 'http://disc.static.szse.cn/download'
    
    url = 'http://www.szse.cn/api/disc/announcement/annList'
    payload = {"seDate":["",""],"stock":["000001"],"channelCode":["listedNotice_disc"],"bigCategoryId":["010301"],"pageSize":30,"pageNum":1}
    headers = {'content-type': 'application/json',
              'Host': 'www.szse.cn',
    'Origin': 'http://www.szse.cn',
    'Referer': 'http://www.szse.cn/disclosure/listed/notice/index.html',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'      
              }
    payload['stock']=[code]
    payload['pageNum']=1
    r = requests.post(url, json=payload)
    ret = r.json()
    count = ret['announceCount']
    data_list = ret['data']
    # 由于限制是30每页，总数超过30，要第二次请求
    if(count>30):
        payload['pageNum']=2
        r = requests.post(url, json=payload)
        ret = r.json()
        data_list.extend(ret['data'])
        
    year_report_map={}
    for d in data_list:
        fileName = d['title']
        if not checkIsChsYearReport(fileName) :
            continue
        # 深交所的年报年份在标题上，如果不能提取出数字，表示错误
        num_list = re.findall('\d{4}',fileName)
        if len(num_list)<1:
            continue
        year = num_list[0]
        if code+'_'+year in year_report_map:
            continue
        year_report_map[code+'_'+year]=download_url_prefix+d['attachPath']
#         print(d['secName'][0],d['title'],d['attachPath'])
    return year_report_map


get_url_by_code_szse('300099')
        

In [ ]:
from datetime import datetime,timedelta
import json
# 上海证券交易所
def get_url_by_code_sse(code):
    url = 'http://query.sse.com.cn/security/stock/queryCompanyStatementNew.do'
    params={
        'jsonCallBack': 'jsonpCallback59529',
        'isPagination': 'true',
        'productId': '600795',
        'keyWord': '',
        'isNew': '1',
        'reportType2': 'DQBG',
        'reportType': 'YEARLY',
        'beginDate': '2002-04-13',
        'endDate': '2019-04-13',
        'pageHelp.pageSize': 25,
        'pageHelp.pageCount': 50,
        'pageHelp.pageNo': 1,
        'pageHelp.beginPage': 1,
        'pageHelp.cacheSize': 1,
        'pageHelp.endPage': 5,
        '_': 1555167217524
    }
    headers = {'Referer': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
    data_list = []
    params['productId']=code
    beginDate=datetime.strptime('2003-01-01','%Y-%m-%d')
    _endDate= datetime.now()
    _3years = timedelta(days=365*3-1)
    while beginDate< _endDate:
        params['beginDate']=beginDate.strftime('%Y-%m-%d')
        beginDate += _3years
        params['endDate']=beginDate.strftime('%Y-%m-%d')
        r = requests.get(url,headers=headers,params=params)
        json_str = r.text[r.text.index('(')+1:-1]
        json_obj = json.loads(json_str)
        data_list.extend(json_obj['result'])


    year_report_map={}
    for d in data_list:
        fileName = d['title']
        if not checkIsChsYearReport(fileName) :
            continue
        if code+'_'+d['bulletin_Year'] in year_report_map:
            continue
#         print(d['bulletin_Year'],d['title'],d['URL'])
        year_report_map[code+'_'+d['bulletin_Year']]='http://www.sse.com.cn'+d['URL']
        
    return year_report_map


get_url_by_code_sse('900929')

In [ ]:

import os
import pandas as pd
import re
from tqdm import tqdm

df1 = pd.read_excel('研究数据0410.xlsx',index_col=0,
            dtype={'Symbol': str})

df1.head()


In [ ]:
df2 = df1[['Symbol','EndDate']] 
data_list = df2.to_dict('records')
code_year_map={}
for d in data_list:
    code = d['Symbol']
    if not re.match('\d{6}',code):
        continue
    if code not in code_year_map:
        code_year_map[code]=[]
    code_year_map[code].append(d['EndDate'][:4])


In [ ]:
"""
下载文件的url
以格式 保存
code 年份 url格式保存
"""
import time
import json
keys = list(code_year_map.keys())
with open('pdf_urls.json') as json_file:  
    pdf_urls = json.load(json_file)

counter=0
for code in tqdm(keys):
    if code+'_2017' in pdf_urls:
        print(f'{code} already download urls,jump.....')
        continue
    counter+=1
    times=0
    while times<=3:
        times += 1
        try:
            print(f'down code for {code}...')
            #获取文件的url 沪市股票买卖的代码是以600或601打头,B股买卖的代码是以900打头
            if code[0]=='6' or code[0]=='9':
                year_report_map=get_url_by_code_sse(code)
            else:
                year_report_map=get_url_by_code_szse(code)
        except Exception as e:
            print('get year_report_map error:',e)
            time.sleep(3)
            continue
        print(f'down code for {code}, result size={len(year_report_map.keys())},keys={year_report_map.keys()}')
        for k in year_report_map.keys():
            pdf_urls[k] = year_report_map[k]
        break
        
    if counter % 20 == 0:
        print('*'*40,'write json file')
        with open('pdf_urls.json','w') as json_file:  
            json.dump(pdf_urls,json_file, sort_keys=True, indent=2)
    
print('*'*40,'write json file')
with open('pdf_urls.json','w') as json_file:  
    json.dump(pdf_urls,json_file, sort_keys=True, indent=2)
        

In [ ]:
# # checkPdf(pdf_dir+'000011_2004'+'.pdf')
# # checkPdf()
# pdfFileReader = PdfFileReader('../scraping/pdf/000006_2003.pdf',strict = False)

# if pdfFileReader.isEncrypted:
#     pdfFileReader.decrypt('')
# # # 获取 PDF 文件的文档信息
# documentInfo = pdfFileReader.getDocumentInfo()
# print('documentInfo = %s' % documentInfo)
# # # 获取页面布局
# # pageLayout = pdfFileReader.getPageLayout()
# # print('pageLayout = %s ' % pageLayout)

# # # 获取页模式
# # pageMode = pdfFileReader.getPageMode()
# # print('pageMode = %s' % pageMode)

# # xmpMetadata = pdfFileReader.getXmpMetadata()
# # print('xmpMetadata  = %s ' % xmpMetadata)

# # 获取 pdf 文件页数
# pageCount = pdfFileReader.getNumPages()

# print('pageCount = %s' % pageCount)